In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Input, Model
from keras.layers import Dense, Dropout, Concatenate,Reshape, Conv2DTranspose,Conv2D, MaxPooling2D, LeakyReLU, UpSampling2D, Lambda
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import segmentation_models as sm
import os
import cv2

Segmentation Models: using `keras` framework.


In [2]:
training_path= "D:\\M\\Exposure_dataset\\training"
validation_path= "D:\\M\\Exposure_dataset\\validation"

In [3]:
image_height= 128
image_width= 128


In [4]:
data_generator = ImageDataGenerator(
    rescale=1.0 / 255, 
)

In [5]:
train_data_generator = data_generator.flow_from_directory(
    training_path,
    target_size=(image_height, image_width),
    batch_size=8,
    class_mode='input', 
    shuffle=True
)

val_data_generator = data_generator.flow_from_directory(
    validation_path,
    target_size=(image_height, image_width),
    batch_size=8,
    class_mode='input',  
    shuffle=False  
)

Found 7070 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [6]:
sm.set_framework('tf.keras')
sm.framework()

'tf.keras'

In [7]:
BACKBONE = 'vgg16'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [8]:
model = sm.Unet(BACKBONE, encoder_weights='imagenet')

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, None, None, 6 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, None, None, 6 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [10]:
epochs = 20
steps_per_epoch = len(train_data_generator)
validation_steps = len(val_data_generator)

In [11]:
from tensorflow.keras.applications import VGG16
vgg = VGG16(weights='imagenet', include_top=False)


In [12]:
# vgg.summary(())

In [13]:
def perceptual_loss(y_true, y_pred):
    layer_name = 'block5_conv3'
    intermediate_layer_model = Model(inputs=vgg.input, outputs=vgg.get_layer(layer_name).output)
    return keras.losses.MSE(intermediate_layer_model(y_true), intermediate_layer_model(y_pred))

In [14]:
with tf.device('/CPU:0'):
    model.compile(optimizer='adam', loss= perceptual_loss, metrics=['mae'])
    history = model.fit(
            train_data_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_data=val_data_generator,
            validation_steps=validation_steps
        )
model.save('weights.h5')

Epoch 1/20


ValueError: in user code:

    C:\Anaconda\envs\deep_learning\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Toqaa\AppData\Local\Temp\ipykernel_1384\947920500.py:4 perceptual_loss  *
        return keras.losses.MSE(intermediate_layer_model(y_true), intermediate_layer_model(y_pred))
    C:\Anaconda\envs\deep_learning\lib\site-packages\keras\engine\base_layer.py:1037 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    C:\Anaconda\envs\deep_learning\lib\site-packages\keras\engine\functional.py:414 call
        return self._run_internal_graph(
    C:\Anaconda\envs\deep_learning\lib\site-packages\keras\engine\functional.py:550 _run_internal_graph
        outputs = node.layer(*args, **kwargs)
    C:\Anaconda\envs\deep_learning\lib\site-packages\keras\engine\base_layer.py:1020 __call__
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Anaconda\envs\deep_learning\l

In [ ]:
# with tf.device('/CPU:0'):
#     H = model.fit(X_train, y_train,
#                         validation_data=(X_val, y_val),
#                         batch_size=8, epochs=100)
#     model.save("weights.h5")